# Description

**TODO**

# Modules

In [1]:
library(reticulate)
pd <- import("pandas")

# Settings

In [2]:
# reference panel
REFERENCE_PANEL <- "GTEX_V8"
# REFERENCE_PANEL = "1000G"

# prediction models
## mashr
EQTL_MODEL <- "MASHR"

# ## elastic net
# EQTL_MODEL = "ELASTIC_NET"

chromosome <- NULL

In [3]:
# Parameters
chromosome = 13
REFERENCE_PANEL = "GTEX_V8"
EQTL_MODEL = "MASHR"


In [4]:
paste0("Using reference panel: ", REFERENCE_PANEL)

[1] "Using reference panel: GTEX_V8"

In [5]:
paste0("Using eQTL model: ", EQTL_MODEL)

[1] "Using eQTL model: MASHR"

In [6]:
# chromosome must be provided as parameter
stopifnot(!is.null(chromosome))

# Paths

In [7]:
GENE_CORRS_DIR <- Sys.getenv("PHENOPLIER_PHENOMEXCAN_LD_BLOCKS_GENE_CORRS_DIR")
IRdisplay::display(GENE_CORRS_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs"

In [8]:
INPUT_DIR <- file.path(GENE_CORRS_DIR, tolower(REFERENCE_PANEL), tolower(EQTL_MODEL), "by_chr")
IRdisplay::display(INPUT_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr"

In [9]:
INPUT_FILE <- file.path(INPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(INPUT_FILE)
stopifnot(file.exists(INPUT_FILE))

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/gene_corrs-chr13.pkl"

In [10]:
OUTPUT_DIR <- file.path(INPUT_DIR, "corrected_positive_definite")
IRdisplay::display(OUTPUT_DIR)
dir.create(OUTPUT_DIR, recursive = TRUE, showWarnings = FALSE)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite"

In [11]:
OUTPUT_FILE <- file.path(OUTPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(OUTPUT_FILE)
if (file.exists(OUTPUT_FILE)) {
  IRdisplay::display("Output file exists, it will be overwritten")
}

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite/gene_corrs-chr13.pkl"

[1] "Output file exists, it will be overwritten"

# Functions

In [12]:
# taken from https://www.r-bloggers.com/2013/08/correcting-a-pseudo-correlation-matrix-to-be-positive-semidefinite/
# TODO: add documentation
CorrectCM <- function(CM, p = 0) {
  n <- dim(var(CM))[1L]
  E <- eigen(CM)
  CM1 <- E$vectors %*% tcrossprod(diag(pmax(E$values, p), n), E$vectors)
  Balance <- diag(1 / sqrt(diag(CM1)))
  CM2 <- Balance %*% CM1 %*% Balance
  return(CM2)
}

In [13]:
is_positive_definite <- function(data) {
  eigenvalues <- eigen(data)$values
  nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]

  if (length(nonpositive_eigenvalues) > 0) {
    IRdisplay::display("We need to correct the data and make the matrix positive definite")
    return(FALSE)
  } else {
    IRdisplay::display("Matrix is already positive definite!")
    return(TRUE)
  }
}

# Load data

In [14]:
gene_corrs <- pd$read_pickle(INPUT_FILE)

In [15]:
dim(gene_corrs)

[1] 118 118

In [16]:
head(gene_corrs[1:10, 1:10])

,ENSG00000121741,ENSG00000121743,ENSG00000121742,ENSG00000032742,ENSG00000150457,ENSG00000150459,ENSG00000102678,ENSG00000127863,ENSG00000228741,ENSG00000102699
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000121741,1.0000000,-0.22188490,0.28655233,-0.29098516,0.13150986,-0.16469326,0.10857837,-0.08381103,0.11537650,0.20402379
ENSG00000121743,-0.2218849,1.00000000,-0.10744722,0.01719924,0.04569570,-0.07201835,-0.04281808,0.02108838,0.04091491,0.08729703
ENSG00000121742,0.2865523,-0.10744722,1.00000000,-0.12306800,-0.01391817,0.09450781,0.07516002,0.02725667,0.04682944,0.02888592
ENSG00000032742,-0.2909852,0.01719924,-0.12306800,1.00000000,-0.06568635,0.22581883,0.04330683,0.08589275,-0.05199242,0.24303508
ENSG00000150457,0.1315099,0.04569570,-0.01391817,-0.06568635,1.00000000,0.01545811,0.03153566,-0.03660881,-0.02789781,-0.10716158
ENSG00000150459,-0.1646933,-0.07201835,0.09450781,0.22581883,0.01545811,1.00000000,0.07763706,0.08202611,0.11398417,-0.09139341


# Check positive definiteness

In [17]:
is_positive_definite(gene_corrs)

[1] "We need to correct the data and make the matrix positive definite"

[1] FALSE

In [18]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs)$values

In [19]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 3

[1] -0.0414812 -0.8963748 -1.6650259

In [20]:
if (length(eigenvalues[eigenvalues <= 0]) == 0) { quit() }

# Make matrix positive definite if needed

In [21]:
gene_corrs_corrected <- CorrectCM(gene_corrs, 1)

In [22]:
dimnames(gene_corrs_corrected)[[1]] <- rownames(gene_corrs)

In [23]:
dimnames(gene_corrs_corrected)[[2]] <- colnames(gene_corrs)

In [24]:
gene_corrs_corrected <- as.data.frame(gene_corrs_corrected)

In [25]:
dim(gene_corrs_corrected)

[1] 118 118

# Check positive definiteness of corrected matrix

In [26]:
is_positive_definite(gene_corrs_corrected)

[1] "Matrix is already positive definite!"

[1] TRUE

In [27]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs_corrected)$values

In [28]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 0

numeric(0)

In [29]:
stopifnot(length(eigenvalues[eigenvalues <= 0]) == 0)

In [30]:
# quick and visual comparison of the two matrices
IRdisplay::display(head(gene_corrs[1:10, 1:10]))
IRdisplay::display(head(gene_corrs_corrected[1:10, 1:10]))

,ENSG00000121741,ENSG00000121743,ENSG00000121742,ENSG00000032742,ENSG00000150457,ENSG00000150459,ENSG00000102678,ENSG00000127863,ENSG00000228741,ENSG00000102699
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000121741,1.0000000,-0.22188490,0.28655233,-0.29098516,0.13150986,-0.16469326,0.10857837,-0.08381103,0.11537650,0.20402379
ENSG00000121743,-0.2218849,1.00000000,-0.10744722,0.01719924,0.04569570,-0.07201835,-0.04281808,0.02108838,0.04091491,0.08729703
ENSG00000121742,0.2865523,-0.10744722,1.00000000,-0.12306800,-0.01391817,0.09450781,0.07516002,0.02725667,0.04682944,0.02888592
ENSG00000032742,-0.2909852,0.01719924,-0.12306800,1.00000000,-0.06568635,0.22581883,0.04330683,0.08589275,-0.05199242,0.24303508
ENSG00000150457,0.1315099,0.04569570,-0.01391817,-0.06568635,1.00000000,0.01545811,0.03153566,-0.03660881,-0.02789781,-0.10716158
ENSG00000150459,-0.1646933,-0.07201835,0.09450781,0.22581883,0.01545811,1.00000000,0.07763706,0.08202611,0.11398417,-0.09139341


,ENSG00000121741,ENSG00000121743,ENSG00000121742,ENSG00000032742,ENSG00000150457,ENSG00000150459,ENSG00000102678,ENSG00000127863,ENSG00000228741,ENSG00000102699
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000121741,1.00000000,-0.09299443,0.142221797,-0.13037638,0.03982904,-0.076648805,0.068653673,-0.058030071,0.051194263,0.05572432
ENSG00000121743,-0.09299443,1.00000000,-0.095686038,0.04508362,-0.01147714,-0.020530275,-0.036196492,0.009276360,-0.009557806,0.01198877
ENSG00000121742,0.14222180,-0.09568604,1.000000000,-0.06846617,0.01795800,0.007002156,0.064252273,-0.008038022,0.045895617,0.02298963
ENSG00000032742,-0.13037638,0.04508362,-0.068466170,1.00000000,-0.06123325,0.094337781,-0.005689510,0.054314875,-0.029243026,0.08524265
ENSG00000150457,0.03982904,-0.01147714,0.017958000,-0.06123325,1.00000000,-0.017771527,0.003024925,-0.023375644,0.007611270,-0.04988472
ENSG00000150459,-0.07664880,-0.02053027,0.007002156,0.09433778,-0.01777153,1.000000000,0.037221997,0.053042993,0.036397161,-0.02154999


Both matrices should "look" similar. We are not interested in perfectly accurate correlation values (they are already inaccurate).

# Save

In [31]:
py_save_object(gene_corrs_corrected, OUTPUT_FILE)